# Задание 2. Итерационные методы решения линейных систем


In [1]:
from latexifier import latexify
from IPython.display import display, Markdown, Latex
import numpy as np
import pandas as pd
from typing import Union, Any
import warnings
warnings.filterwarnings('ignore')


In [2]:
class Matrix(np.ndarray):
    """extends np.ndarray"""

    def __new__(cls, input_array):
        obj = np.asarray(input_array).view(cls)
        return obj

    def inverse_rows(self, i: int, j: int):
        """Swaps i-th and j-th rows of a matrix"""
        self[i], self[j] = self[j].copy(), self[i].copy()

    def inverse_cols(self, i: int, j: int):
        """Swaps i-th and j-th columns of a matrix"""
        self = self.transpose()
        self.inverse_rows(i, j)
        self = self.transpose()

    def tex(self) -> str:
        """Latexifies the matrix rounding up values up to 6 decimal values"""
        return latexify(self)

    def norm(self) -> float:
        """Calculates Matrix norm using `np.linalg.norm` with parameter `np.inf`"""
        return np.linalg.norm(self, np.inf)

    def inv(self) -> Any:
        return Matrix(np.linalg.inv(self))


def append(M: np.ndarray, N: np.ndarray) -> Matrix:
    """Appends two numpy arrays with axis=1"""
    return Matrix(np.append(arr=np.asarray(M), values=N, axis=1))


In [3]:
def Gauss(A: Matrix, B: Matrix, n: int) -> Matrix:
    """
    Вычисляет решения линейного уравнения Ax = B методом Гаусса с выбором
    главного элемента по столбцам (с перестановкой строк)
    """
    AB = append(A, B)
    X = [[0] for _ in range(n)]
    for k in range(n):
        j = k
        for i in range(k+1, n):
            if abs(AB[j][k]) < abs(AB[i][k]):
                j = i
        AB.inverse_rows(k, j)
        for j in range(k+1, n):
            c = AB[j][k] / AB[k][k]
            for i in range(n+1):
                AB[j][i] -= c * AB[k][i]
    X[n-1][0] = AB[n-1][n] / AB[n-1][n-1]
    for k in range(n-1, -1, -1):
        s = 0
        for i in range(k+1, n):
            s += AB[k][i] * X[i][0]
        X[k][0] = (AB[k][n] - s) / AB[k][k]
    return Matrix(np.array(X))


In [4]:
def print_tex(*argv) -> None:
    """Displays a LaTex Markdown output"""
    res = ' '.join(['$$'] + [(latexify(arg) if isinstance(arg,
                   np.ndarray) else str(arg)) for arg in argv] + ['$$'])
    display(Markdown(res))


def get_latex_column(arg: str, n: int) -> str:
    """
    Prepares a string with LaTex syntax for a column of `arg` with indeces form 1 to `n`
    """
    res = [r"""\begin{pmatrix}""", "\n"]
    for i in range(n):
        line = arg + "_{" + str(i+1) + "}" + (r"\\" if i != n-1 else "") + "\n"
        res.append(line)
    res.append(r"""\end{pmatrix}""")
    return ''.join(res)


def get_exact_column(col: Matrix, n: int) -> str:
    """Prepares a string with LaTex syntax for a column of {col} elements"""
    res = [r"""\begin{pmatrix}""", "\n"]
    for i in range(n):
        line = str(float(col[i][0])) + (r"\\" if i != n-1 else "") + "\n"
        res.append(line)
    res.append(r"""\end{pmatrix}""")
    return ''.join(res)


def print_gauss_output(A: Matrix, B: Matrix, X: Matrix, n: int, name: str) -> None:
    """Oupputs the result of solving a linear equation"""
    B_ = A @ X
    print_tex(r'\text{', name, '}~X^* = ',
              get_latex_column("x^*", n), '=', get_exact_column(X, n))
    print_tex(rf'A \times X^* = ', A.tex(), X.tex(), '=', B_.tex(), '= B^*')
    print_tex('B = ', get_exact_column(B, n),
              r'\stackrel{?}{=}', get_exact_column(B_, n), '= B^*')
    print_tex(r'B - B^* = ', get_exact_column(B - B_, n))


## Вариант 12


In [21]:
n = 4
A = Matrix(np.random.rand(n, n))
B = Matrix(np.random.rand(n, 1))
for i in range(n):
    # для метода верхней релаксации матрица должна быть симметричной
    for j in range(n):
        A[i][j] = A[j][i]
    # создание диагонального преобладания
    A[i][i] += np.sum([A[i][j] for j in range(n) if i != j])
print_tex('A=', A, ',~~', 'B=', B)


$$ A= \begin{pmatrix} 1.463568 & 0.460030 & 0.114581 & 0.828541 \\ 0.460030 & 1.548733 & 0.256708 & 0.290546 \\ 0.114581 & 0.256708 & 2.122415 & 0.845903 \\ 0.828541 & 0.290546 & 0.845903 & 2.119560 \end{pmatrix} ,~~ B= \begin{pmatrix} 0.269234 \\ 0.817609 \\ 0.557398 \\ 0.761909 \end{pmatrix} $$

In [22]:
eigenvals = Matrix(np.linalg.eigvals(A))
print('Собственные числа матрицы A:', eigenvals)

Собственные числа матрицы A: [3.39332241 0.69405063 1.78471402 1.38218885]


Найдем решение $x^*$ методом Гаусса.


In [23]:
X_Gauss = Gauss(A, B, n)
print_gauss_output(A, B, X_Gauss, n, 'Решение методом Гаусса:')


$$ \text{ Решение методом Гаусса: }~X^* =  \begin{pmatrix}
x^*_{1}\\
x^*_{2}\\
x^*_{3}\\
x^*_{4}
\end{pmatrix} = \begin{pmatrix}
-0.16205374720237917\\
0.5021316458298258\\
0.08271510192261151\\
0.32097031289681593
\end{pmatrix} $$

$$ A \times X^* =  \begin{pmatrix} 1.463568 & 0.460030 & 0.114581 & 0.828541 \\ 0.460030 & 1.548733 & 0.256708 & 0.290546 \\ 0.114581 & 0.256708 & 2.122415 & 0.845903 \\ 0.828541 & 0.290546 & 0.845903 & 2.119560 \end{pmatrix} \begin{pmatrix} -0.162054 \\ 0.502132 \\ 0.082715 \\ 0.320970 \end{pmatrix} = \begin{pmatrix} 0.269234 \\ 0.817609 \\ 0.557398 \\ 0.761909 \end{pmatrix} = B^* $$

$$ B =  \begin{pmatrix}
0.26923381627412846\\
0.8176086289860681\\
0.5573980815716444\\
0.761908802235568
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
0.26923381627412846\\
0.817608628986068\\
0.5573980815716444\\
0.7619088022355679
\end{pmatrix} = B^* $$

$$ B - B^* =  \begin{pmatrix}
0.0\\
1.1102230246251565e-16\\
0.0\\
1.1102230246251565e-16
\end{pmatrix} $$

In [24]:
def compare_result(name: str, val: Matrix, n: int = n) -> None:
    print_tex(name, '=', get_exact_column(val, n), r'\stackrel{?}{=}', get_exact_column(X_Gauss, n), '=X^*')
    print_tex(rf'||X^* - {name}||_\infty=', (X_Gauss - val).norm())

data = {}

def add_data(method_name: str, iterations: int, ans: Matrix) -> None:
    data[method_name] = {
        'Количество Итераций': str(iterations),
        'Фактическая Погрешность': str((X_Gauss - ans).norm())
    }

Преобразуем исходную систему к системе вида $x = H_Dx + g_D$
$$ H_D = E - D^{-1}A $$
$$ g_D = D^{-1}b $$


In [25]:
D, D_inv = Matrix(np.zeros((n, n))), Matrix(np.zeros((n, n)))

for i in range(n):
    D[i][i] = A[i][i]
    D_inv[i][i] = 1 / A[i][i]

H_D = Matrix(np.identity(n) - D_inv @ A)
g_D = Matrix(D_inv @ B)
print_tex('H_D=', H_D, ',~~', 'g_D=', g_D)
print_tex(rf'||H_D||_\infty = ', H_D.norm())


$$ H_D= \begin{pmatrix} 0 & -0.314321 & -0.078289 & -0.566111 \\ -0.297036 & 0 & -0.165753 & -0.187602 \\ -0.053986 & -0.120951 & 0 & -0.398557 \\ -0.390903 & -0.137078 & -0.399094 & 0 \end{pmatrix} ,~~ g_D= \begin{pmatrix} 0.183957 \\ 0.527921 \\ 0.262625 \\ 0.359466 \end{pmatrix} $$

$$ ||H_D||_\infty =  0.9587207648235021 $$

Найдем априорную оценку для того $k$, при котором $||x^* - x^k||_\infty < \varepsilon,~\varepsilon = 0.001$


In [26]:
def apr_eval(
    H: Matrix = H_D,
    g: Matrix = g_D,
    k: int = 1
) -> float:
    return H.norm()**k / (1 - H.norm()) * g.norm()


eps = 1.e-3
k = 1
while (apr_eval(k=k) >= eps):
    k += 1
print('Количество итераций для достижения нужной точности по априорной оценке:', k)


Количество итераций для достижения нужной точности по априорной оценке: 225


In [27]:
def iterative_method(
    H: Matrix = H_D,
    G: Matrix = g_D,
    n: int = n
) -> Union[Matrix, Matrix, int]:
    next = Matrix(np.zeros((n, 1)))
    iter = 0
    while (X_Gauss - next).norm() >= eps:
        prev, next = next, H @ next + G
        iter += 1
    return next, prev, iter


iter_ans, iter_prev, iter = iterative_method()
compare_result(rf'X^{({iter})}', iter_ans)


$$ X^{25} = \begin{pmatrix}
-0.16119988315779954\\
0.502745043360571\\
0.0832593790881056\\
0.3217609422760341
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
-0.16205374720237917\\
0.5021316458298258\\
0.08271510192261151\\
0.32097031289681593
\end{pmatrix} =X^* $$

$$ ||X^* - X^{25}||_\infty= 0.0008538640445796275 $$

In [28]:
add_data('Метод простой итерации', iter, iter_ans)

In [29]:
print(f'Априорная оценка при k={iter}:', apr_eval(k=iter))

Априорная оценка при k=25: 4.458004500414823


In [30]:
def apostr_eval(
    x_k: Matrix = iter_ans,
    x_prev: Matrix = iter_prev,
    H: Matrix = H_D,
) -> float:
    return H.norm() / (1 - H.norm()) * (x_k - x_prev).norm()

print('Апостериорная оценка:', apostr_eval(iter_ans, iter_prev))


Апостериорная оценка: 0.04450548980949765


In [31]:
def spec_radius(H: Matrix = H_D):
    return max(np.absolute(np.linalg.eigvals(H)))


def luster_approx(x_k: Matrix, x_prev: Matrix, H: Matrix = H_D) -> Matrix:
    return x_prev + (x_k - x_prev) / (1 - spec_radius(H))


X_luster = luster_approx(iter_ans, iter_prev, H_D) # погрешность ухудшается, так не должно быть

compare_result(rf'X^{({iter})}_\text{{lust}}', X_luster)


$$ X^{25}_\text{lust} = \begin{pmatrix}
-0.15346042066026888\\
0.5083174536929084\\
0.08819303057130905\\
0.32894925423939964
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
-0.16205374720237917\\
0.5021316458298258\\
0.08271510192261151\\
0.32097031289681593
\end{pmatrix} =X^* $$

$$ ||X^* - X^{25}_\text{lust}||_\infty= 0.008593326542110286 $$

Вычислим решение $x = H_Dx + g_D$ методом Зейделя с точностью $\varepsilon = 0.001$

In [33]:
def seid_method(
    H: Matrix = H_D,
    G: Matrix = g_D,
    n: int = n
) -> Union[Matrix, Matrix, int]:
    H_L = Matrix(np.array([[H[i][j] if i > j else 0 for j in range(n)] for i in range(n)]))
    H_R = Matrix(np.array([[H[i][j] if i <= j else 0 for j in range(n)] for i in range(n)]))
    H_seid = (np.identity(n) - H_L).inv() @ H_R
    g_seid = (np.identity(n) - H_L).inv() @ G
    next = Matrix(np.zeros((n, 1)))
    iter = 0
    while (X_Gauss - next).norm() >= eps:
        prev, next = next, H_seid @ next + g_seid
        iter += 1
    return next, prev, iter
    
seid_ans, seid_prev, seid_iter = seid_method()
compare_result(rf'X^{({seid_iter})}_\text{{seid}}', seid_ans)

$$ X^{7}_\text{seid} = \begin{pmatrix}
-0.16131076178267362\\
0.5019327724268661\\
0.0831714745650917\\
0.3205250039087184
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
-0.16205374720237917\\
0.5021316458298258\\
0.08271510192261151\\
0.32097031289681593
\end{pmatrix} =X^* $$

$$ ||X^* - X^{7}_\text{seid}||_\infty= 0.0007429854197055497 $$

In [34]:
add_data('Метод Зейделя', seid_iter, seid_ans)

Получим решение системы $Ax = b$ методом верхней релаксации с точностью $\varepsilon = 0.001$

In [35]:
def rel_method(
    A: Matrix = A,
    b: Matrix = B,
    H: Matrix = H_D,
    n: int = n
) -> Union[Matrix, Matrix, int]:
    rho = spec_radius(H)
    q_opt = 2 / (1 + np.sqrt(1 - rho**2)) - 0.15
    next = Matrix(np.zeros((n, 1)))
    iter = 0
    while (X_Gauss - next).norm() >= eps:
        prev = next
        for i in range(n):
            next[i][0] = prev[i][0] + q_opt * (b[i][0] - np.sum([next[j][0] * A[i][j] for j in range(
                i)]) - np.sum([prev[j][0] * A[i][j] for j in range(i, n)])) / A[i][i]
        iter += 1
    return next, prev, iter


rel_ans, rel_prev, rel_iter = rel_method()
compare_result(rf'X^{({rel_iter})}_\text{{rel}}', rel_ans)

$$ X^{5}_\text{rel} = \begin{pmatrix}
-0.16128600907594584\\
0.5020818214747548\\
0.08316593525810918\\
0.3205866345845041
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
-0.16205374720237917\\
0.5021316458298258\\
0.08271510192261151\\
0.32097031289681593
\end{pmatrix} =X^* $$

$$ ||X^* - X^{5}_\text{rel}||_\infty= 0.0007677381264333261 $$

In [36]:
add_data('Метод верхней релаксации', rel_iter, rel_ans)

In [37]:
df = pd.DataFrame(data)
df

,Метод простой итерации,Метод Зейделя,Метод верхней релаксации
Количество Итераций,25,7,5
Фактическая Погрешность,0.0008538640445796275,0.0007429854197055497,0.0007677381264333261
